# **Basket and Recipes**
This script tries to match each basket in the dataset with the corresponding cuisine style

### **Import** section

In [2]:
from multiprocessing import Pool
from random import sample
import numpy as np
import pandas as pd
import itertools
import json
import ast
from statistics import mean 

---
### **Natural Language Toolkit**
#### Used for matching items in the basket dataset and recipes dataset

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')

ps = PorterStemmer()

# This function is used to convert a string into another similar string
# By removing plurals, stop words, punctuation, and so on...
def itemParser(s):
    
    s = s.replace("\n", "")
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(r"(\d)", "", s)
    s = re.sub(r'\([^)]*\)', '', s)
    s = re.sub(u'\w*\u2122', '', s)
    s = s.lower()

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)

    filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
    s = ' '.join(filtered_sentence)
    
    return s

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/filippotessaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/filippotessaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---
### **Utility** function

In [13]:
def jaccard_sim(s1, s2):
    return len(s1.intersection(s2))/len(s1.union(s2))

In [14]:
def getRecipeByID(rec, idr):
    data = rec[rec.id.eq(idr)]
    return (ast.literal_eval(data.new_ingredients.values[0]), data.cuisine.values[0])

---

### **DataSet** Reading

In [15]:
baskets = []
with open('../Data/groceries_norm.csv', 'r') as f:
    for line in f:
        basket = []
        items = line.split(",")
        for item in items:
            #parsed = itemParser(item)
            basket.append(item)
        basket = set(basket)
        baskets.append(basket)

In order to make the computation faster, we take a sample (10%) of the whole recipes dataset

In [16]:
keep = 0.1
recipes = pd.read_csv('../Data/train_norm.csv')
sample_recipes = recipes.sample(n=int(len(recipes)*keep))

---
### **Parsing** Functions

In [17]:
def parse_ingredients(string_vector):    
    # Compact and remove inner quote symbols
    string_vector = string_vector.replace("', '", ',')    
    # Remove brakets, first and last quote symbols
    string_vector = string_vector[2:-2]    
    # Tokenize and make a set
    string_vector = set(string_vector.split(","))    
    return string_vector

In [18]:
def parse_recipes_dataset(recipes):
    parsed_recipes = []
    for recipe in recipes.itertuples(index=True, name='Pandas'):
        cuisine = getattr(recipe, "cuisine")
        ID = getattr(recipe, "id")
        ingredients = parse_ingredients(getattr(recipe, "new_ingredients"))
        parsed_recipes.append((cuisine, ID, ingredients))        
    return parsed_recipes

---

### **Matching**

In [19]:
def match(basket, recipes):

    # valid is the array containing all those recipes having a positive similarity
    valid = []
    
    # For all the (sampled) recipes in the dataset
    for recipe in recipes:
        
        # Get the data
        cuisine     = recipe[0]
        ID          = recipe[1]
        ingredients = recipe[2]
        
        # Compute the similarity
        sim = jaccard_sim(ingredients, basket)
        if sim > 0:
            valid.append((float(sim), cuisine, ID))
            
    basket = list(basket)
    
    return (basket, valid)

In [20]:
# In order to work properly, the dataset must be parsed (convert string ingredients to lists)
parsed_dataset = parse_recipes_dataset(recipes)

In [21]:
# Wrapper
def matching(basket):
    return match(basket, parsed_dataset)

cores = 6
with Pool(cores) as pool:
    return_values = pool.map(matching, baskets)    
    best_assoc = []
    for value in return_values:
        best_assoc.append(value)

---
### **Cleaning and Saving**

In [22]:
max_basket_size = 0
for basket in baskets:
    if len(basket) > max_basket_size:
        max_basket_size = len(basket)
print("Max Basket Size: " + str(max_basket_size))

Max Basket Size: 32


In [23]:
output = pd.DataFrame(best_assoc)
output.columns = ["Basket", "Recipes"]

In [24]:
display(output)

,Basket,Recipes
0,"[citru fruit, readi soup\n, semifinish bread, ...","[(0.05, irish, 31027), (0.1, french, 18643), (..."
1,"[coffe\n, tropic fruit, yogurt]","[(0.125, french, 36148), (0.1111111111111111, ..."
2,[whole milk\n],[]
3,"[cream chees, yogurt, pip fruit, meat spread\n]","[(0.09090909090909091, southern_us, 17610), (0..."
4,"[whole milk, condens milk, long life bakeri pr...","[(0.1, mexican, 25164), (0.05, moroccan, 27858..."
...,...,...
9830,"[root veget, sausag, flour, chocol, beef, coff...","[(0.03571428571428571, filipino, 20130), (0.02..."
9831,[cook chocol\n],[]
9832,"[veget, domest egg, frozen dessert, rollsbun, ...","[(0.047619047619047616, filipino, 20130), (0.0..."
9833,"[bottl water, semifinish bread, soda, bottl be...","[(0.07142857142857142, indian, 18452), (0.0555..."


In [53]:
def save_assoc_raw(assoc_list, fname):
    with open(fname, 'w') as out_file:
        json.dump(assoc_list, out_file)
        out_file.close()

In [54]:
def save_assoc_pickle(pd_assoc_list, fname):
    pd_assoc_list.to_pickle(fname)

In [55]:
def load_assoc_raw(fname):
    with open(fname, 'r') as in_file:
        content = json.load(in_file)
        in_file.close()
        return content

In [56]:
def load_assoc_pickle(fname):
    return pd.read_pickle(fname)

In [40]:
save_assoc_raw(best_assoc, '../Data/raw_assoc_basket_recipes.txt')
raw = load_assoc_raw('../Data/raw_assoc_basket_recipes.txt')

In [27]:
save_assoc_pickle(output, '../Data/basket_recipes.pkl')
pickle = load_assoc_pickle('../Data/basket_recipes.pkl')

In [42]:
display(test2)

,Basket,Recipes
0,"[margarin, citru fruit, readi soup, semifinish...","[(0.05, irish, 31027), (0.1, french, 18643), (..."
1,"[yogurt, tropic fruit, coff]","[(0.125, french, 36148), (0.1111111111111111, ..."
2,[whole milk],"[(0.14285714285714285, mexican, 25164), (0.058..."
3,"[yogurt, pip fruit, meat spread, cream chee]",[]
4,"[long life bakeri product, conden milk, veget,...","[(0.1, mexican, 25164), (0.05, moroccan, 27858..."
...,...,...
19665,"[napkin, salti snack, whippedsour cream, hambu...","[(0.03571428571428571, filipino, 20130), (0.02..."
19666,[cook chocol],[]
19667,"[veget, rum, cling filmbag, citru fruit, butte...","[(0.047619047619047616, filipino, 20130), (0.0..."
19668,"[bottl beer, bottl water, soda, semifinish bread]","[(0.07142857142857142, indian, 18452), (0.0555..."


---

### **Sorting Recipes by Similarity**

In [25]:
sorted_assoc = []
for association in best_assoc:
    
    basket = association[0]
    detail = association[1]
    
    # Sort detail with respect to the similarity
    detail = sorted(detail, key=lambda x: x[0], reverse=True)
    sorted_assoc.append((basket, detail))
    

In [26]:
output = pd.DataFrame(sorted_assoc)
output.columns = ["Basket", "Recipes"]
display(output)

,Basket,Recipes
0,"[citru fruit, readi soup\n, semifinish bread, ...","[(0.14285714285714285, mexican, 3500), (0.1428..."
1,"[coffe\n, tropic fruit, yogurt]","[(0.125, french, 36148), (0.1111111111111111, ..."
2,[whole milk\n],[]
3,"[cream chees, yogurt, pip fruit, meat spread\n]","[(0.2, brazilian, 42490), (0.16666666666666666..."
4,"[whole milk, condens milk, long life bakeri pr...","[(0.2857142857142857, indian, 37190), (0.25, f..."
...,...,...
9830,"[root veget, sausag, flour, chocol, beef, coff...","[(0.19047619047619047, southern_us, 38721), (0..."
9831,[cook chocol\n],[]
9832,"[veget, domest egg, frozen dessert, rollsbun, ...","[(0.15384615384615385, italian, 6539), (0.1428..."
9833,"[bottl water, semifinish bread, soda, bottl be...","[(0.16666666666666666, southern_us, 46178), (0..."


In [1]:
clustered = pd.read_pickle("../Data/Clustered_Basket.pkl")

NameError: name 'pd' is not defined

In [28]:
output["K_Means"] = pd.Series(clustered["Cluster"])
#output["DB_Scan"] = pd.Series(clustered["DB_Scan"])

In [29]:
display(output)

,Basket,Recipes,K_Means
0,"[citru fruit, readi soup\n, semifinish bread, ...","[(0.14285714285714285, mexican, 3500), (0.1428...",1
1,"[coffe\n, tropic fruit, yogurt]","[(0.125, french, 36148), (0.1111111111111111, ...",5
2,[whole milk\n],[],2
3,"[cream chees, yogurt, pip fruit, meat spread\n]","[(0.2, brazilian, 42490), (0.16666666666666666...",5
4,"[whole milk, condens milk, long life bakeri pr...","[(0.2857142857142857, indian, 37190), (0.25, f...",7
...,...,...,...
9830,"[root veget, sausag, flour, chocol, beef, coff...","[(0.19047619047619047, southern_us, 38721), (0...",9
9831,[cook chocol\n],[],1
9832,"[veget, domest egg, frozen dessert, rollsbun, ...","[(0.15384615384615385, italian, 6539), (0.1428...",6
9833,"[bottl water, semifinish bread, soda, bottl be...","[(0.16666666666666666, southern_us, 46178), (0...",4


In [31]:
final = []
for o in output.values:
    basket = o[0]
    
    recipes = o[1]
    
    if len(recipes) > 0:
        tmp = {}
        for recipe in recipes:
            if recipe[1] not in tmp.keys():
                tmp[recipe[1]] = []
            tmp[recipe[1]].append(recipe[0])

        for item in tmp:
            tmp[item] = len(tmp[item]) * mean(tmp[item])

        tmp = {k: v for k, v in sorted(tmp.items(), key=lambda item: item[1], reverse=True)}
        best = (list(tmp.keys())[0], tmp[list(tmp.keys())[0]])

        cuisine = best[0]
        score = best[1]
    
    else:
        cuisine = None
        score = 0
        
    
    kmeans = o[2]
    dbs = o[2]
    
    final.append([basket, cuisine, score, kmeans, dbs])

In [38]:
best_recipes = pd.DataFrame(final)
best_recipes.columns = ["Basket", "Cuisine", "Confidence", "K_Means", "DB_Scan"]
display(best_recipes)

,Basket,Cuisine,Confidence,K_Means,DB_Scan
0,"[citru fruit, readi soup\n, semifinish bread, ...",southern_us,4.545730,1,1
1,"[coffe\n, tropic fruit, yogurt]",french,0.125000,5,5
2,[whole milk\n],None,0.000000,2,2
3,"[cream chees, yogurt, pip fruit, meat spread\n]",mexican,15.189903,5,5
4,"[whole milk, condens milk, long life bakeri pr...",southern_us,15.935106,7,7
...,...,...,...,...,...
9830,"[root veget, sausag, flour, chocol, beef, coff...",southern_us,72.550579,9,9
9831,[cook chocol\n],None,0.000000,1,1
9832,"[veget, domest egg, frozen dessert, rollsbun, ...",southern_us,74.624349,6,6
9833,"[bottl water, semifinish bread, soda, bottl be...",southern_us,0.752915,4,4


In [39]:
best_recipes.to_pickle("../Data/Best_recipes.pkl")

---
### **Analysis**

In [171]:
#sorted_assoc = load_assoc_raw('../Data/raw_assoc_basket_recipes.txt')

In [58]:
newds= best_recipes.groupby(["K_Means", "Cuisine"])["Confidence"].mean()
newds = pd.DataFrame(newds)
display(newds.query('Confidence > 25'))
newds.query('Confidence > 25').to_csv('../Images/confidence-cousine.csv')

Confidence
K_Means Cuisine                
0       indian        30.689124
        italian       35.046646
        mexican       72.162342
        southern_us   26.645967
1       mexican       33.373935
        southern_us   52.554934
2       indian        35.279785
        italian       45.889978
        mexican       68.139299
        southern_us   27.948098
3       mexican       29.526656
        southern_us   51.217669
4       mexican       28.351445
5       italian       32.916431
        mexican       28.155230
        southern_us   45.952298
6       mexican       42.153536
        southern_us   68.549109
7       indian        31.515750
        italian       42.086229
        mexican       65.931159
        southern_us   34.740179
9       italian       49.149045
        mexican       50.340808
        southern_us   38.999575

In [40]:
grouped_recipes = best_recipes.groupby(["K_Means", "Cuisine"]).count().reset_index()

display(grouped_recipes)

,K_Means,Cuisine,Basket,Confidence,DB_Scan
0,0,indian,14,14,14
1,0,italian,49,49,49
2,0,mexican,13,13,13
3,0,southern_us,322,322,322
4,1,british,2,2,2
...,...,...,...,...,...
88,9,chinese,7,7,7
89,9,indian,22,22,22
90,9,italian,91,91,91
91,9,mexican,41,41,41


In [41]:
grouped_recipes = grouped_recipes.iloc[:,:3]

In [42]:
x = {}
for row in grouped_recipes.values:
    if row[0] not in x.keys():
        x[row[0]] = []
    x[row[0]].append([row[1], row[2]])

In [45]:
for cluster in x:
    cuisines = x[cluster]
    maxv = [None, -1]
    for c in cuisines:
        if maxv[1] < c[1]:
            maxv = [c[0], c[1]]
    print(maxv)

['southern_us', 322]
['italian', 360]
['southern_us', 928]
['italian', 143]
['southern_us', 385]
['southern_us', 114]
['chinese', 318]
['southern_us', 286]
['southern_us', 128]
['southern_us', 262]


In [43]:
single_recipes = []
for basket in sorted_assoc:
    if len(basket[1]) == 1:
        bask = basket[0]
        
        # Get data
        cuisine = basket[1][0][1]
        rec_id = basket[1][0][2]
        
        det = (cuisine, rec_id)
        single_recipes.append((bask, det))

In [44]:
single_recipes

[(['rollsbun', 'bottl water', 'frankfurt'], ('french', 13111)),
 (['dessert', 'frankfurt'], ('french', 13111)),
 (['rollsbun', 'frankfurt'], ('french', 13111)),
 (['rollsbun', 'frankfurt'], ('french', 13111)),
 (['rollsbun', 'frankfurt', 'fruitveget juic', 'bottl water', 'hygien articl'],
  ('french', 13111)),
 (['rollsbun', 'frankfurt'], ('french', 13111)),
 (['frankfurt', 'yogurt'], ('french', 13111)),
 (['frankfurt'], ('french', 13111)),
 (['rollsbun', 'frankfurt'], ('french', 13111)),
 (['rollsbun', 'frankfurt'], ('french', 13111)),
 (['bottl beer', 'frankfurt'], ('french', 13111)),
 (['frozen fruit', 'deterg'], ('jamaican', 34911)),
 (['frozen meal', 'shop bag', 'semifinish bread', 'frankfurt'],
  ('french', 13111)),
 (['rollsbun', 'pip fruit', 'bottl water', 'frankfurt'], ('french', 13111)),
 (['hamburg meat', 'frankfurt'], ('french', 13111)),
 (['fruitveget juic', 'rollsbun', 'frankfurt'], ('french', 13111)),
 (['frankfurt'], ('french', 13111)),
 (['domest egg', 'frankfurt'], ('

In [45]:
getRecipeByID(recipes, 13111)

(['pork loin',
  'champagn',
  'frankfurt',
  'garlic clove',
  'sauerkraut',
  'brat',
  'knockwurst',
  'ground black pepper',
  'salt pork',
  'onion'],
 'french')